#### Copie de Mehdi - ne pas enregistrer dessus (génération de conflits)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from glob import glob
import pickle

import numpy as np
import pandas as pd

import nltk
import re
import codecs
import unidecode
#pip install unidecode
import mpld3
# pip install mpld3
import stop_words
# pip install stop-words
from nltk import SnowballStemmer, pos_tag, word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import HashingVectorizer,TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import *
from sklearn.semi_supervised import *

In [60]:
" ANZOANfr tknernd,o/E;id,d;zo,".lower()

' anzoanfr tknernd,o/e;id,d;zo,'

** Lecture des données ** 

Objectif garder uniquement les parties contenant les compétences

#### Suppression des sauts de ligne

In [11]:
import string, re

In [94]:
def del_line_feed(s):  
    """Delete \n in the text"""
    regex = re.compile('[%s]' % '(\\n)*(\\x0c)*(\\uf0a7)*')
    return regex.sub(' ', s)

#### Suppression ponctuation

In [95]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [96]:
#le maintient de la ponctuation pertube le stop words, apostrophe gérée dans text_treatment
regex = re.compile('[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~')) 

def del_punct(s):  
    """Delete punctuation in the text"""
    return regex.sub(' ', s)

In [97]:
class CV:
    def __init__(self, filename, title, cv, doc_type):
        self.filename = filename
        self.title = title
        self.cv = cv
        self.doc_type = doc_type

In [98]:
def ao_delete_context_part(ao):
    """Supprime la partie de l'appel d'offre consacrée au contexte pour ceux qui suivent le pattern SNCF"""
    split_ao = ao.split('DESCRIPTION DE LA MISSION')
    if len(split_ao)>1:
        ao_light = split_ao[1] 
    else :
        ao_light = split_ao[0]
    return ao_light

In [99]:
def ao_delete_contact_part(ao):
    """Supprime la partie de l'appel d'offre consacrée au contact et adresse pour 
    ceux suivant le même pattern"""
    
    split1_ao = ao.split('conditions d’execution')
    if len(split1_ao)>1:
        if 'autres points' in split1_ao[1]:
            split2_ao = split1_ao[1].split('autres points')
            ao_light = split1_ao[0] + " " + split2_ao[1]
            
        elif 'profils et competences attendus' in split1_ao[1] :
            split2_ao = split1_ao[1].split('profils et competences attendus')
            ao_light = split1_ao[0] + " " + split2_ao[1]
            
        else:
            ao_light = split1_ao[0]
    
    else :
        ao_light = split1_ao[0]
        
    return ao_light

In [100]:
def ao_delete_element_reponse(ao):
    if "elements de reponse attendus" in ao:
        ao_light = ao.split("elements de reponse attendus")[0]
    elif "methode d'attribution" in ao:
        ao_light = ao.split("methode d'attribution")[0]
    else:
        ao_light = ao
    return ao_light

In [101]:
def cv_keep_comp(cv):
    if "compétences & outils" in cv:
        cv_split1 = cv.split("compétences & outils")[1]
        if "formations & certifications" in cv_split1:
            cv_light = cv_split1.split("formations & certifications")[0]
        else:
            cv_light = ""
            
    elif "compétences" in cv:
        cv_split1 = cv.split("compétences")[1]
        if "experience professionnelle" in cv_split1:
            cv_light = cv_split1.split("experience professionnelle")[0]
        elif "expérience professionnelle" in cv_split1:
            cv_light = cv_split1.split("expérience professionnelle")[0]
        elif "parcours professionnel" in cv_split1:
            cv_light = cv_split1.split("parcours professionnel")[0]
        elif "langues" in cv_split1:
            cv_light = cv_split1.split("langues")[0]
        elif "experience" in cv_split1:
            cv_light = cv_split1.split("experience")[0]
        elif "expérience" in cv_split1:
            cv_light = cv_split1.split("expérience")[0]
        else:
            cv_light = ""
            
    elif "competences" in cv:
        cv_split1 = cv.split("competences")[1]
        if "experience" in cv_split1:
            cv_light = cv_split1.split("experience")[0]
        elif "expérience" in cv_split1:
            cv_light = cv_split1.split("expérience")[0]
        elif "parcours professionnel" in cv_split1:
            cv_light = cv_split1.split("parcours professionnel")[0]
        elif "formations & certifications" in cv_split1:
            cv_light = cv_split1.split("formations & certifications")[0]
        elif "formation" in cv_split1:
            cv_light = cv_split1.split("formations & certifications")[0]
        elif "langues" in cv_split1:
            cv_light = cv_split1.split("langues")[0]
        else:
            cv_light = ""
    
    elif "it skills" in cv:
        cv_split1 = cv.split("it skills")[1]
        if "activities & interests" in cv_split1:
            cv_light = cv_split1.split("activities & interests")[0]
        elif "hobbies" in cv_split1:
            cv_light = cv_split1.split("hobbies")[0]
        else:
            cv_light = ""
    
    else:
        cv_light = ""
    
    return cv_light 

In [102]:
#download cv in a list
def load_Doc_list(nombre, path, doc_type):
    liste_paths = [path+directory for directory in os.listdir(path)]
    #print(liste_paths)
    liste_cv = []
    dico_cv = {}
    for path in liste_paths :
        cv = CV(0, 0, 0, 0)
        cv.filename = os.path.basename(path)
        
        try:
            if doc_type == 'ao':
                cv.cv = open(path).read()
                cv.cv = del_punct(del_line_feed(cv.cv.lower()))
                cv.cv = ao_delete_element_reponse(ao_delete_context_part(ao_delete_contact_part(cv.cv)))

            else :
                cv.cv = open(path).read()
                cv.cv = cv_keep_comp(del_punct(del_line_feed(cv.cv.lower())))
                
            cv.doc_type = doc_type
            liste_cv.append(cv.cv)
            dico_cv[cv.filename.split(".")[0]] = cv#.
        except UnicodeDecodeError:
            print("Error !")
            print(path)
    return liste_cv, dico_cv

In [103]:
print("Loading titles")
def load_title_list(nombre, path):
    path = "../Maha/data/"
    dico_titles = {}
    filenames = sorted(glob(os.path.join(path,"*.csv")))
    for file in filenames[:nombre]:
        with open(file, encoding='latin-1') as csvfile:
            readCSV = csv.reader(csvfile, delimiter=';')
            for row in readCSV:
                dico_titles[row[0]] = row[1]
    return dico_titles

Loading titles


In [104]:
path_AO = '../Commun/Data_Talan/txt_ao/'
path_CV = '../Commun/Data_Talan/txt/'

In [105]:
liste_cv, dico_cv = load_Doc_list(200, path_CV, 'cv')
liste_AO, dico_AO = load_Doc_list(200, path_AO, 'ao')

dico_global = dict(dico_cv)
dico_global.update(dico_AO)

In [106]:
liste_cv[0]

'  savoir faire  gestion de la relation client   conseil   gestion de portefeuille de clients professionnels   techniques de planification   gestion de projets   marketing informatique logiciels de gestion  erp  crm  bureautique  pack office  word  excel  access  powerpoint   '

In [107]:
len(dico_global)

691

** Reconnaissance du langage du CV**

In [108]:
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens] #from text get list of word in minuscule

    
    for language in stopwords.fileids(): # pour chaque langue
        stopwords_set = set(stopwords.words(language)) #je mets les stop words du langage dans un set
        words_set = set(words) #je mets les mots de mon texte dans un set
        #je prends l'intersection entre les mots de mon texte et les mots du stopwords dans le langage donné
        common_elements = words_set & stopwords_set
        
        #je compute mon score comme le nombre d'éléments en communs dictionnaire [langage : score]
        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

In [109]:
import nltk
nltk.download('stopwords')
stopwords.fileids()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehdiregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [110]:
def get_cv_langue(liste_cv) :
    """Return resume witten in the specified language in parameter"""
    liste_2 = []
    for cv in liste_cv:
        if max(_calculate_languages_ratios(cv),key =_calculate_languages_ratios(cv).get)=='french':
            liste_2.append(cv)
    return liste_2

In [113]:
liste_cv_fr = get_cv_langue(liste_cv) 
liste_ao_fr = get_cv_langue(liste_AO)

** Preprocessing du text **

In [114]:
def text_treatment (text):
    text = text.lower()
    text = text.replace("\x00", '').replace("\x01", '').replace("\x02", '').replace("\x03", '') \
    .replace("\x04", '').replace("\x05", '').replace("\x06", '').replace("\x07", '').replace("\x08", '') \
    .replace("\x0e", '').replace("\x11", '').replace("\x12", '').replace("\x10", '').replace("\x19", '') \
    .replace("\x1b", '').replace("\x14", '').replace("\x15", '').replace('/', '').replace('=', '').replace("〓", "") \
    .replace("»", "").replace("«", "").replace("¬", "").replace('`', '').replace (" -", "").replace("•", "")\
    .replace("l'", "").replace("l’", "").replace("l´", "").replace("d’", "").replace("d'", "").replace("d´","")\
    .replace("j’", "").replace("j'", "").replace("j´","").replace("n’", "").replace("n'", "").replace("n´","")\
    .replace("”", "").replace("~", "").replace("§", "").replace("¨", "").replace("©", "").replace("›", "")\
    .replace("₋", "").replace("→", "").replace("⇨", "").replace("∎", "").replace("√", "").replace("□", "")\
    .replace("*", "").replace("&", "").replace("►", "").replace("◊", "").replace("☞", "").replace("#", "")\
    .replace("%", "").replace("❖", "").replace("➠", "").replace("➢", "").replace("", "").replace("✓", "") \
    .replace("√", "").replace("✔", "").replace("♦", "").replace("◦", "").replace("●", "").replace("▫", "")\
    .replace("▪", "").replace("…", "").replace("þ", "").replace("®", "").replace('', '').replace("...", "")\
    .replace(" o ", "")
    text = unidecode.unidecode(text) # remove accent
    return text

** Gestion des stop words **

In [115]:
#generate stopwords
stop_words_py = set(stop_words.get_stop_words('french'))

# attention certains stop words pourraient être utiles par la suite
stopwords_set_manuel = set(["an", "ans", 'les', 'moins', 'd\'un','janvier', 'fevrier', 'février', 'mars', 'avril', \
                 'mai', 'juin', 'juillet', 'aout', 'août', 'septembre', 'octobre', 'novembre', 'décembre', \
                  'decembre', 'moins', 'mise', 'universit\xc3\xa9', 'université', 'universite', 'ion','sage', \
                  'o', 'rac', 'vers', 'via', 'p\xc3\xa9rim\xc3\xa8tre', 'périmètre','et','paris','x',"\x00",\
                          "\x01","\x02", "\x03","\x04","\x05","\x06","\x07","\x08","\x09","\x0e","\x0e","\x11",\
                           "\x12","\x13","\x14","\x15","\x16","\x17","\x18","\x19","transport","puis","lieu",\
                           "adresse","entre",'dun','dune','chez','boulognebillancourt','bt','etc','recrutement','main',\
                           'and', 'paie','paiement','environ','place','france','paris','mois','mobile','mobiles',\
                           'nanterre','source','sources','concerne','concernant','of','non','notes','rh','minimum',\
                           'maximum','bac','site','sites','actuellement','telephone','telephonique','telephoniques','ca','demenager',\
                           'demenagement','participer','participation','lycee','baccalaureat','lien','liens','in',\
                           'indeed','email','indeedcomrd7e8913ed00d0384','aujourhui','afin','toujours','enterprise',\
                           "guide","10g","11g","9i",'ad','v10','v2','v3','v5','v6','v8','v9','pablo','neruda',\
                           'dec','stelsia','cid1','sens','va','24h24', '7j7', 'levalloisperret','louis', 'armand', 'ermont',\
                            'localisation','anne', 'perot',' s ','mission', 'date', 'debut','fin','avenue', 'val', 'oise',\
                           'echeant','bout','division','prestations', 'intellectuelles', 'contexte','descroix','24h', '7j',\
                           'disposition','places','moyens', 'sncf', 'mis','rue','levallois','perret','scnf','ainsi',\
                           "jusqu'au","jusqu'a",'ast','exercant','souhaitee','sectorielles','sectorielles','competences',\
                           "ensemble","phases","phase","technique","garantir","coherence","referentiels","server","langages","langage"\
                           "secteurs",'activite','metier','campaign',"sup'com",'collegiate','contest','deuxieme',\
                            'competition','sectorielles','outils','arts','metiers','methodes','methode','trace',\
                           'entity','services','centres','interet','musculation','cuisine','voyage','travail','groupe','realisation',\
                            'team','building','presentation','devant','publique','versions','anterieurs','etats',\
                            'crystal','reports','superviseur','tiers','secteurs','marche','tertiaire','specialite',\
                           'divers','lumiere','lyon','parcours','sise','iup','institut','universitaire','professionnel',\
                           '1ere','annee','3eme','annee','licence','dut','niort','techniques','langues','vivantes','anglais',\
                            'professionnel','espagnol','scolaire','arabe','parle','revue','code','check','list','2008r2',\
                           'biginsight','general','prog','procedurale','parallele','qt','compilation','courant','jour',\
                           'informatiques','suites','active','activites','adaptation','alternance','animation','annees',\
                           'appliquees','autour','aupres','autres','avance','avancees','bien','campus','cartographie','cas',\
                           'center','centrale','centre','certification','certifications','charges','choix','classe','client',\
                            'clients','com','comite','comites','cours','derive','detail','differents','diplomes',\
                           'e','ecrit','ecriture','effets','engagee','enjeux','enquete','entite','entreprise',\
                           'esprit','etant','etoile','europe','evenements','externes','famille','fixes','fortes','fr',\
                           'garantie','generale','grandes','group','hp','humaines','impact','import','investissement',\
                           'langage','lille','livraison','locaux','mises','missions','mode','national','nationale','necessaires',\
                            'niveau','passage','plusieurs','periode','permettra','pertinence','principaux','principes','promotion',\
                            'proposes','rapidement','rencontres','reprise','saint','salle','secteur','sein','selon',\
                           'suite','suivant','tant','tendances','toutes','travaux','tres','type',\
                            'utilisateur','utilisateurs','utilisation','utilises','voix','voyages',])
stop_words_main = stop_words_py | stopwords_set_manuel
stop_words_main = list(stop_words_main)
print("taille stop words liste : ", len(stop_words_main))

taille stop words liste :  611


In [116]:
stop_words_main

['fus',
 'deuxieme',
 'donc',
 'sommes',
 '24h',
 'lumiere',
 '7j7',
 'o',
 'puis',
 '\x0e',
 'voyage',
 'minimum',
 'neruda',
 '10g',
 'en',
 'votre',
 'division',
 'concerne',
 'pour',
 'eût',
 'differents',
 'v10',
 'travaux',
 '\x01',
 'rapidement',
 'eussiez',
 'ast',
 'levalloisperret',
 'participation',
 'revue',
 'depuis',
 'aujourhui',
 'ainsi',
 'auraient',
 'nationale',
 'état',
 'du',
 'ayez',
 'qu',
 'etats',
 'mois',
 'deux',
 'se',
 'armand',
 'date',
 'national',
 'devrait',
 'garantir',
 'dès',
 'val',
 'bon',
 'devriez',
 'novembre',
 'arabe',
 'ad',
 'animation',
 'main',
 'aussi',
 'juin',
 'étais',
 'cartographie',
 'enjeux',
 'cid1',
 'mes',
 'suites',
 'périmètre',
 't',
 'ayant',
 'tes',
 'il',
 'aie',
 'cette',
 '\x14',
 'qt',
 '11g',
 'au',
 'esprit',
 'concernant',
 'avoir',
 'ma',
 'ne',
 'sera',
 'v3',
 '3eme',
 'devrez',
 'participer',
 'v5',
 'iup',
 '24h24',
 'eusses',
 'nommés',
 'principaux',
 'ses',
 'été',
 'd',
 'car',
 'debut',
 'vont',
 'intellect

In [117]:
#voir si utile
def remove_stopwords(text,stopwords_list):
    text_temp = " ".join(text.split())+" "
    for word in stopwords_list:
        text_temp = text_temp.replace(" "+word+" ", " ")
    return text_temp

In [119]:
liste_cv_clean = [remove_stopwords(text_treatment(cv),stop_words_main) for cv in liste_cv_fr]
liste_ao_clean = [remove_stopwords(text_treatment(ao),stop_words_main) for ao in liste_ao_fr]

In [120]:
#SnowballStemmer 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

In [121]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [122]:
liste_cv_tokenized = [tokenize_only(cv) for cv in liste_cv_clean]
liste_ao_tokenized = [tokenize_only(ao) for ao in liste_ao_clean]

In [124]:
#get set
liste_flat = []
for row in liste_cv_tokenized:
    liste_flat += row
set_cv = set(liste_flat)
print("longueur set cv {}".format(len(set_cv)))

#get set
liste_flat = []
for row in liste_ao_tokenized:
    liste_flat += row
set_ao = set(liste_flat)
print("longueur set ao {}".format(len(set_ao)))

#get set
set_intersect = set_cv & set_ao
print("longueur set intersection {}".format(len(set_intersect)))


longueur set cv 4784
longueur set ao 2227
longueur set intersection 1045


--------------